# Examples of noise added to the simulations

This is a copy of the super basic token economy, used to demonstrate the noise add-ons.

In [1]:
import numpy as np
import scipy
import sys
import os
tokenlab_path=os.path.abspath("").replace('notebooks','src')
sys.path.insert(0,tokenlab_path)

from TokenLab.simulationcomponents import *
from TokenLab.simulationcomponents.usergrowthclasses import *
from TokenLab.simulationcomponents.transactionclasses import *
from TokenLab.simulationcomponents.tokeneconomyclasses import *
from TokenLab.simulationcomponents.transactionclasses import *
from TokenLab.simulationcomponents.agentpoolclasses import *
from TokenLab.simulationcomponents.pricingclasses import *
from TokenLab.simulationcomponents.addons import AddOn_RandomNoise, AddOn_RandomNoiseProportional
from utils.helpers import *
from matplotlib import pyplot as plt

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


IndentationError: expected an indented block (transactionclasses.py, line 594)

# Globals

We start by defining our globals. We choose 60 iterations, with each iteration representing 1 month, and an average holding time of 1 month. The total circulating supply is 100 million

In [ ]:
ITERATIONS=60
HOLDING_TIME=1
SUPPLY=10**9
INITIAL_PRICE=0.1

# Increasing user growth without noise

Let's see what user growth looks like when we don't apply any noise add-ons.

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space)


ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=100,currency='$')
plt.plot(usm_fiat._num_users_store)

This is what the price looks like below. The price growth is equivalent to user growth.

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

In [ ]:
reps

# Increasing user growth with noise

Now let's apply some noise to the user growth. We see that the price now takes a more realistic trajectory.

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space,
                           noise_addon=AddOn_RandomNoise(dist_params={'loc':5000,'scale':2000}))


ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=10,currency='$')
plt.plot(usm_fiat._num_users_store)

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

# Increasing user growth with adaptive noise

Now let's apply some proportional noise. The variance scales in accordance with user growth. In this example it is 10% of the total number of users. This is calculated upon the original user projections.

We see that the price varies more wildly compared to before.

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space,
                           noise_addon=AddOn_RandomNoiseProportional(std_param=10))


ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=1000,currency='$')
plt.plot(usm_fiat._num_users_store)
plt.ylabel('num users')
plt.xlabel('month')

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

# Probabilistic active users

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space,
                           noise_addon=[AddOn_RandomNoiseProportional(std_param=10),
                                        AddOn_RandomReduction(reduction_dist: scipy.stats.rv_continuous = scipy.stats.uniform(loc=0, scale=0.1))])


ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=1000,currency='$')
plt.plot(usm_fiat._num_users_store)
plt.ylabel('num users')
plt.xlabel('month')

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

# User growth stochastic and and stochastic transactions

Now let's try to add everything all together! In this case we add the noise to user growth, while also used the stochastic transactions class. The stochastic transactions class does not accept any noise addons, because of the way it's designed, as it implements multiple stochastic components simultaneously.

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space,
                           noise_addon=AddOn_RandomNoiseProportional(std_param=10))

tsm_fiat=TransactionManagement_Stochastic(activity_probs=np.linspace(0.25,1,ITERATIONS),
                                            value_dist_parameters={'loc':1000,'scale':200},
                                           transactions_dist_parameters={'mu':1})

ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=tsm_fiat,currency='$')

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

# User growth stochastic and linear transaction trend with noise component
Now let's try to add everything all together! In this case we add the noise to user growth, while also used the stochastic transactions class. The stochastic transactions class does not accept any noise addons, because of the way it's designed, as it implements multiple stochastic components simultaneously.

In [ ]:
usm_fiat=UserGrowth_Spaced(100,54000,ITERATIONS,log_saturated_space,
                           noise_addon=AddOn_RandomNoiseProportional(std_param=10))

tsm_fiat=TransactionManagement_Trend(average_transaction_initial=200,average_transaction_final=1500,
                                     noise_addon=AddOn_RandomNoiseProportional(std_param=4),num_steps=ITERATIONS)

ap_fiat=AgentPool_Basic(users_controller=usm_fiat,transactions_controller=tsm_fiat,currency='$')
plt.plot(tsm_fiat._transactions_means_store)
plt.xlabel('average size of transaction')
plt.ylabel('month')

In [ ]:
te=TokenEconomy_Basic(holding_time=HOLDING_TIME,supply=SUPPLY,token='tokenA',initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot

# Price plus noise

There are different pricing functions that we can use. Let's try replacing the normal pricing function, with one that adds Gaussian noise and let's see what happens.

In [ ]:
te=TokenEconomy_Basic(holding_time=HoldingTime_Stochastic(),supply=SUPPLY,
                      price_function=PriceFunction_EOE,token='tokenA',
                      price_function_parameters={'noise_addon':AddOn_RandomNoise()},initial_price=INITIAL_PRICE)
te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot
reps

# Price plus noise plus adaptive variance

This is a special pricing mechanism which induces randomness in the price, but the variance of the noise is proportional to the price. The idea behind this is similar to https://www.investopedia.com/terms/g/garch.asp and has also been observed in the cryptocurrency markets. For example, this holds true for Bitcoin. The higher the value of bitcoin, the higher the observed volatility.

In [ ]:
te=TokenEconomy_Basic(holding_time=HoldingTime_Stochastic(),supply=SUPPLY,
                      price_function=PriceFunction_EOE,token='tokenA',
                      price_function_parameters={'noise_addon':AddOn_RandomNoiseProportional(std_param=7)},
                      initial_price=INITIAL_PRICE)

te.add_agent_pools([ap_fiat])
meta=TokenMetaSimulator(te)
meta.execute(iterations=ITERATIONS,repetitions=50)
reps=meta.get_data()
plot,data=meta.get_timeseries('tokenA_price')
plot